In [2]:
# Import libraries

import pandas as pd
import pyspark
from pyspark.sql.functions import *
from pydataset import data

import re

# Create spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()

### 1. Create a spark data frame that contains your favorite programming languages.

1. The name of the column should be language
1. View the schema of the dataframe
1. Output the shape of the dataframe
1. Show the first 5 records in the dataframe

In [6]:
# Create a pandas dataframe first

df = pd.DataFrame(['Python', 'JavaScript', 'Java', 'C#', 'C', 'C++', 'Go', 'R', 'Swift', 'PHP'], 
                  columns = ['language'])

# Conver the pandas dataframe to spark dataframe
df = spark.createDataFrame(df)

# View the schema of the spark dataframe
df.printSchema()

# Output the shape of the dataframe
print("DataFrame shape:", df.count(), " x ", len(df.columns))

# Show the first 5 records in the dataframe
df.show(5)

root
 |-- language: string (nullable = true)

DataFrame shape: 10  x  1
+----------+
|  language|
+----------+
|    Python|
|JavaScript|
|      Java|
|        C#|
|         C|
+----------+
only showing top 5 rows



### 2. Load the `mpg` dataset as a spark dataframe.

In [8]:
# Load the mpg dataset as a spark dataframe
mpg = spark.createDataFrame(data('mpg'))

# Print the schema
mpg.printSchema()

# Print the first 3 rows
mpg.show(3)

root
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- displ: double (nullable = true)
 |-- year: long (nullable = true)
 |-- cyl: long (nullable = true)
 |-- trans: string (nullable = true)
 |-- drv: string (nullable = true)
 |-- cty: long (nullable = true)
 |-- hwy: long (nullable = true)
 |-- fl: string (nullable = true)
 |-- class: string (nullable = true)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 3 rows



### 2-a. Create 1 column of output that contains a message like the one below:
`The 1999 audi a4 has a 4 cylinder engine.`

In [19]:
mpg.select(
    concat(
        lit("The "), 
        mpg.year,
        lit(" "), 
        mpg.manufacturer,
        lit(" "), 
        mpg.model, 
        lit(" has a "), 
        mpg.cyl, 
        lit(" cylinder engine") 
    ).alias("vehicle_cylinder_desc")
).show(5, truncate=False)

+----------------------------------------+
|vehicle_cylinder_desc                   |
+----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine|
|The 1999 audi a4 has a 4 cylinder engine|
|The 2008 audi a4 has a 4 cylinder engine|
|The 2008 audi a4 has a 4 cylinder engine|
|The 1999 audi a4 has a 6 cylinder engine|
+----------------------------------------+
only showing top 5 rows



### 2-b. Transform the trans column so that it only contains either manual or auto.
- This can be done in many ways:
    - `regexp_extract`
    - `regexp_replace`
    - `when`

In [20]:
mpg.select(col("trans")).show(5)

+----------+
|     trans|
+----------+
|  auto(l5)|
|manual(m5)|
|manual(m6)|
|  auto(av)|
|  auto(l5)|
+----------+
only showing top 5 rows



In [45]:
mpg.select(
    col("trans"), 
    regexp_extract("trans", r"^([a-z]*)", 1).alias("transmission"), 
    regexp_extract("trans", r"^(\w+)\(", 1).alias("regexp_extract"),
    regexp_replace("trans", r"\(.+$", "").alias("regexp_replace"),
    when(mpg.trans.like("auto%"), "auto") # Let us specify a value to produce under certain condition
    .otherwise("manual").alias("when + like")
    
).show()

+----------+------------+--------------+--------------+-----------+
|     trans|transmission|regexp_extract|regexp_replace|when + like|
+----------+------------+--------------+--------------+-----------+
|  auto(l5)|        auto|          auto|          auto|       auto|
|manual(m5)|      manual|        manual|        manual|     manual|
|manual(m6)|      manual|        manual|        manual|     manual|
|  auto(av)|        auto|          auto|          auto|       auto|
|  auto(l5)|        auto|          auto|          auto|       auto|
|manual(m5)|      manual|        manual|        manual|     manual|
|  auto(av)|        auto|          auto|          auto|       auto|
|manual(m5)|      manual|        manual|        manual|     manual|
|  auto(l5)|        auto|          auto|          auto|       auto|
|manual(m6)|      manual|        manual|        manual|     manual|
|  auto(s6)|        auto|          auto|          auto|       auto|
|  auto(l5)|        auto|          auto|        

In [43]:
import re

def trans(string):
    regexp = r'^(\w+)\('
    return re.search(regexp, string)

In [44]:
trans("manual(m5)")

<re.Match object; span=(0, 7), match='manual('>

### Load the `tips` dataset as a spark dataframe

In [5]:
# Load the tips dataset as a spark dataframe

tips = spark.createDataFrame(data('tips'))

# Print the schema
tips.printSchema()

# Print the first 5 rows
tips.show(5)

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: long (nullable = true)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



### 3-a. What percentage of observations are smokers?

In [39]:
# Use roll up

(
    tips.rollup('smoker')
        .count()
        .sort('smoker')
        .select(
            col("smoker"),
            (concat(
                round(col("count")/244*100, 0).cast("int"), 
                lit("%")).alias("percentage")
            )
        )
).show()

+------+----------+
|smoker|percentage|
+------+----------+
|  null|      100%|
|    No|       62%|
|   Yes|       38%|
+------+----------+



In [35]:
# Use groupby

(
    tips.groupby("smoker")
        .count()
        .withColumn(
            "percentage", # Add a new column named 'percentage'
            concat(
                round(col("count") / tips.count() * 100, 0).cast("int"),
                lit("%")
            )
        )
).show()

+------+-----+----------+
|smoker|count|percentage|
+------+-----+----------+
|    No|  151|       62%|
|   Yes|   93|       38%|
+------+-----+----------+



### 3-b. Create a column that contains the tip percentage

In [23]:
tips.select(
    tips.total_bill, 
    tips.tip,
    (tips.tip/tips.total_bill).alias("percentage")
).show(5)

+----------+----+-------------------+
|total_bill| tip|         percentage|
+----------+----+-------------------+
|     16.99|1.01|0.05944673337257211|
|     10.34|1.66|0.16054158607350097|
|     21.01| 3.5|0.16658733936220846|
|     23.68|3.31| 0.1397804054054054|
|     24.59|3.61|0.14680764538430255|
+----------+----+-------------------+
only showing top 5 rows



In [40]:
tips.withColumn(
    "tip_percentage",
    col("tip") / col("total_bill")
).show(5)

+----------+----+------+------+---+------+----+-------------------+
|total_bill| tip|   sex|smoker|day|  time|size|     tip_percentage|
+----------+----+------+------+---+------+----+-------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|0.05944673337257211|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|0.16054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|0.16658733936220846|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 0.1397804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|0.14680764538430255|
+----------+----+------+------+---+------+----+-------------------+
only showing top 5 rows



### 3-c. Calculate the average tip percentage for each combination of sex and smoker.

In [32]:
# groupby alone

(
    tips.groupby(tips.sex, tips.smoker)
        .agg((mean(col("tip")/col("total_bill")).alias("percentage")))
).show()

+------+------+-------------------+
|   sex|smoker|         percentage|
+------+------+-------------------+
|  Male|    No| 0.1606687151291298|
|  Male|   Yes|0.15277117520248512|
|Female|    No| 0.1569209707691836|
|Female|   Yes|0.18215035269941032|
+------+------+-------------------+



In [42]:
# groupby + pivot

(
    tips.withColumn("tip_percentage", tips.tip/tips.total_bill)
    .groupby('sex')
    .pivot("smoker")
    .agg(mean("tip_percentage"))
).show(5)

+------+------------------+-------------------+
|   sex|                No|                Yes|
+------+------------------+-------------------+
|Female|0.1569209707691836|0.18215035269941032|
|  Male|0.1606687151291298|0.15277117520248512|
+------+------------------+-------------------+

